In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Voicebot과 Chatbot 연동을 통해 음성 기반 상담 서비스 구현

In [ ]:
OPENAI_API_KEY= # OpenAI API Key

STT 음성 인식 모델

mock speech file 만들기

In [ ]:
## generate mock speech file

from pathlib import Path
from openai import OpenAI
import os
client = OpenAI(api_key = OPENAI_API_KEY)

save_path = "/content/drive/MyDrive/데이터/ch0603"

query_list = ["오늘 인천에서 뉴욕으로 가는 비행기를 예약하고 싶어요", "항공편 취소를 하려면 어떻게 해야 하나요?", "오후 2시 이후에 출발하는 뉴욕행 비행기를 예매할 수 있나요?"]

for i, query in enumerate(query_list):

  speech_file_path = os.path.join(save_path, f"query_{i}.mp3")
  response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=query
  )
  response.stream_to_file(speech_file_path)

<ipython-input-23-4908b524c2e4>:20: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [ ]:
from IPython.display import Audio, display
display(Audio("/content/drive/MyDrive/데이터/ch0603/query_1.mp3", autoplay=True))

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = OPENAI_API_KEY)

def stt(speech_file_path):
  audio_file = open(speech_file_path, "rb")
  transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
  )
  return transcription.text

print(stt("/content/drive/MyDrive/데이터/ch0603/query_1.mp3"))

항공편 취소를 하려면 어떻게 해야 하나요?


TTS 음성 생성 모델

In [ ]:
import os
from openai import OpenAI
from IPython.display import Audio, display

client = OpenAI(api_key = OPENAI_API_KEY)

class voice_generater():
  def __init__(self, cache_path):
    self.cache_path = cache_path
    self.cache_id = 0

  def generate(self, txt):
    save_path = os.path.join(self.cache_path, f"{self.cache_id}.mp3")
    response = client.audio.speech.create(
      model="tts-1",
      voice="alloy",
      input=txt
    )
    is_save = response.stream_to_file(save_path)

    display(Audio(save_path, autoplay=True))


voice_gen = voice_generater(cache_path="/content/drive/MyDrive/데이터/ch0603")


In [ ]:
voice_gen.generate("무엇을 도와 드릴까요?")

<ipython-input-26-f8ada45edda4>:19: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


##### stt, tts 함수와 chatbot을 연동하여 상담 봇 만들기

In [ ]:
class VoiceChatbot():
  def __init__(self, cache_path, system_prompt, proper_nouns_prompt):
    self.cache_path = cache_path
    self.cache_id = 10
    self.system_prompt = system_prompt
    self.proper_nouns_prompt = proper_nouns_prompt

  def query(self, query_path):
    question = self.stt(query_path)
    answer = self.generate_answer(question)
    answer = self.post_process(answer)
    save_path = self.tts(answer)
    return save_path

  def stt(self, voice_path):
    audio_file = open(voice_path, "rb")
    transcription = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file
    )
    return transcription.text

  def tts(self, txt):
    save_path = os.path.join(self.cache_path, f"{self.cache_id}.mp3")
    response = client.audio.speech.create(
      model="tts-1",
      voice="echo",
      input=txt
    )
    is_save = response.stream_to_file(save_path)

    display(Audio(save_path, autoplay=True))
    self.cache_id += 1

    return save_path

  def generate_answer(self, question):
    chat_completion = client.chat.completions.create(
    messages=[
            {
                "role": "system",
                "content": self.system_prompt,
            },
            {
                "role": "user",
                "content": question,
            }
        ],
        model="gpt-3.5-turbo",
    )
    response_txt = chat_completion.choices[0].message.content

    return response_txt

  def post_process(self, txt):
    return txt.replace("\n", "")


voiceChatbot = VoiceChatbot(
    cache_path="/content/drive/MyDrive/데이터/ch0603",
    system_prompt="너는 항공편 관련 상담을 해주는 인공지능 에이전트야. 사용자의 질문에 대해 친절하게 답변해줘",
    proper_nouns_prompt="도시: 뉴욕, 인천, 런던, 파리"
)

In [ ]:
from IPython.display import Audio, display
query1 = "/content/drive/MyDrive/데이터/ch0603/query_0.mp3"
display(Audio(query1, autoplay=True))

In [ ]:
voiceChatbot.query(query1)

<ipython-input-28-14f132a260f5>:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


'/content/drive/MyDrive/데이터/ch0603/10.mp3'

In [ ]:
from IPython.display import Audio, display
query2 = "/content/drive/MyDrive/데이터/ch0603/query_2.mp3"
display(Audio(query2, autoplay=True))

In [ ]:
voiceChatbot.query(query2)

<ipython-input-28-14f132a260f5>:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


'/content/drive/MyDrive/데이터/ch0603/11.mp3'

In [ ]:
from IPython.display import Audio, display
query3 = "/content/drive/MyDrive/데이터/ch0603/query_1.mp3"
display(Audio(query3, autoplay=True))

In [ ]:
voiceChatbot.query(query3)

<ipython-input-28-14f132a260f5>:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


'/content/drive/MyDrive/데이터/ch0603/12.mp3'

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr

In [ ]:
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

greet("Gangwoo", 5)

'Hello, Gangwoo!!!!!'

In [ ]:
def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ee3debb6cf3191a227.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
demo = gr.Interface(
    voiceChatbot.stt,
    gr.Audio(sources=["microphone"], type="filepath"),
    "text",
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bda1167e9588515d97.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
demo = gr.Interface(
      fn=voiceChatbot.query,
      inputs=gr.Audio(sources="microphone", type="filepath"),
      outputs=gr.Audio(label="Generated Speech", type="filepath"),
  )

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://820e9f5b393367aa89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Reference
https://huggingface.co/learn/audio-course/en/chapter7/speech-to-speech